In [9]:
import sys
path = "../../.."
if path not in sys.path:
    sys.path.insert(0, path)

In [24]:
from data_retrieval import lipade_groundtruth
from data_retrieval.tools.data_loader import getDataLoader
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np
import torch
import clip

sourceText = 'blip2'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
corpus = "lipade_groundtruth"
distancePath = "../results/distance/" + corpus + "/"
rawPath = "../results/raw/" + corpus + "/"

In [11]:
print(clip.available_models())
modelName = 'ViT-L/14'
modelLowerName = "vitl14"

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']


In [ ]:
_,m,_ = lipade_groundtruth.getDataset(mode="similar")

In [21]:
model, preprocess = clip.load(modelName, device)

representations = []
with torch.no_grad():
    for i in tqdm(range(len(m[1])), desc=modelName):
        caption = clip.tokenize(m[1][i]).to(device)
        representations.append(model.encode_text(caption).cpu())

ViT-L/14: 100%|██████████| 279/279 [00:01<00:00, 241.81it/s]


In [22]:
representations = np.array(representations).reshape((len(m[1]),-1))
sim = np.array(cosine_similarity(representations, representations))

distance = 1 - (sim+1)/2
distance -= np.diag(distance)

In [25]:
np.save(distancePath + sourceText + "_text_into_" + "clip_transfer_" + modelLowerName + ".npy", distance)
np.save(rawPath + sourceText + "_text_into_" + "clip_transfer_" + modelLowerName + ".npy", representations)